# Building models

A model is a collection of connected layers that process the inputs to generate the outputs. You can use the nn package to define models. The nn package is a collection of modules that provide common deep learning layers. A module or layer of _nn_ receives input tensors, computes output tensors, and holds the weights, if any. There are two methods we can use to define models in PyTorch: _nn.Sequential_ and _nn.Module_.

In [1]:
# Defining a linear layer

# create a linear layer and print out its output size:

import torch
import torch.nn as nn

# input tensor dimension 64*1000, i.e., our input tensor has 64 features and 1000 observations
input_tensor = torch.randn(64, 1000)

# linear layer with 1000 inputs and 100 outputs
linear_layer = nn.Linear(1000, 100)

# output of the linear layer
output = linear_layer(input_tensor) 
print(output.size())

torch.Size([64, 100])


## Defining models using nn.Sequential

We can use the nn.Sequential package to create a deep learning model by passing layers in order. Consider the two-layer neural network depicted in the following image:

Our network has four nodes as input, five nodes in the hidden layer, and one node as the output

In [2]:
# define a two-layer model
model = nn.Sequential(
    nn.Linear(4, 5),
    nn.ReLU(),
    nn.Linear(5, 1),
)

print(model)

Sequential(
  (0): Linear(in_features=4, out_features=5, bias=True)
  (1): ReLU()
  (2): Linear(in_features=5, out_features=1, bias=True)
)


## Defining models using nn.Module

Another way of defining models in PyTorch is by subclassing the nn.Module class. 
In this method, we specify the layers in the __init__ method of the class. Then, 
in the forward method, we apply the layers to inputs. This method provides better 
flexibility for building customized models.

In [7]:
import torch.nn.functional as F


class Net(nn.Module):

    # defining the __init__ function
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
    

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, (4*4*50))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return F.log_softmax(x, dim=1)



# override both class functions, __init__ and forward

#Net.__init__ = __init__
#Net.forward = forward

In [8]:
# create an object of the Net class and print the model

model = Net()
print(model)

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


# Moving the model to a CUDA device

A model is a collection of parameters. By default, the model will be hosted on the CPU

In [10]:
# get the model's current device

print(next(model.parameters()).device)

cpu


To take advantage of the GPU, we need to move our input tensors and the model itself to the GPU by explicitly using the to() method.

Here, we copy the model to the GPU if PyTorch reports that one is available, or otherwise keep the model on the CPU. 
By using this construction, we can determine whether a GPU is available at the start of our code 

In [11]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")


model.to(device)

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)